In [1]:
import sys
sys.path.append('C:\\Users\\Максимилиан\\Desktop\\Skoltech\\Reinforcement learning\\Final project')
from Dynamics import get_next_state, state_to_coords, get_energy,normalize_angle
from Environment import DoublePendulumEnv, ObservationSpaceCartPole
from PPO.Proximal_Policy_Optimization import PPO, unscaled_action
from PPO.train import train
from PPO.config_PPO import config

In [2]:
from torch import nn
import torch
from torch.utils.tensorboard import SummaryWriter
import gym
from torch.distributions import Normal
from gym.spaces import Box
import random
from IPython.display import clear_output
from torch.distributions import MultivariateNormal
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [4]:
state0 = np.array([0,np.pi/2,np.pi/2,
                0,0,0])
state = state0
state
#max_initial_angle = 3 * 2 * np.pi / 360
max_initial_angle = 0
dt = 0.02

In [5]:
oords = np.array(state_to_coords(state)[:, 0])
oords

array([0., 0.])

In [6]:
get_next_state(state,100,10)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


array([ 57.09394275,  -2.53782625,  -2.52245306,  68.56067285,
         3.83151235, -23.02406187])

# Environment

In [7]:
env = DoublePendulumEnv(init_state=state, dt=config['dt'], max_initial_angle = 0)

Environment initialized


C:\Users\Максимилиан\AppData\Roaming\Python\Python38\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [8]:
env.action_space.shape

(1,)

In [11]:

def train():
    print("============================================================================================")
    # max. timestep per episode. For DoubleCartPoleEnv, time constraint is 200 timesteps. After that environment is
    # reset.
    max_ep_len = config['max_episode_length']
    # The training phase will sample and update for 1 million timestep.
    max_training_steps = int(config['max_training_steps'])

    # In order, to check ongoing progress, average reward is printed at every 10_000 timesteps.
    print_freq = 10_000

    # Saving model parameters at every 1_00_000 timesteps.
    save_model_freq = int(config['save_model_freq'])

    action_std = config['action_std']  # Initial standard deviation.
    action_std_decay_rate = config['action_std_decay_rate']  # Decay rate of standard deviation.
    min_action_std = config['min_action_std']  # Threshold standard deviation.
    action_std_decay_freq = int(2e5)  # Decay the standard deviation every 200000 timesteps

    update_timestep = config['update_timestep']  # set old_policy parameters to new_policy parameters.
    K_epochs = config['max_number_of_epoch']  # Number of epochs before updating old policy parameters.
    eps_clip = config['eps_clip']  # clip range for surrogate loss function.
    gamma = config['gamma']  # Discount factor.

    lr_actor = config['lr_actor']  # Learning rate for optimizer of actor network.
    lr_critic = config['lr_critic']  # Learning rate for optimizer of critic network.
    env_name = 'DoubleInvPendulum'
    print("Training Environment:" + env_name)
    
    
    
    env = DoublePendulumEnv(init_state=state, dt=config['dt'], max_initial_angle = 0)

    observation_shape = env.observation_space.shape[0]  # Observation shape
    action_shape = env.action_space.shape[0]  # Action shape

    directory = "PPO3_Trained"
    if not os.path.exists(directory):
        os.makedirs(directory)

    directory = directory + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)


    checkpoint_path = directory + "PPO3_{}.pth".format(env_name)


    print("--------------------------------------------------------------------------------------------")
    print("PPO update frequency : " + str(update_timestep) + " timesteps")
    print("PPO K epochs : ", K_epochs)
    print("PPO epsilon clip : ", eps_clip)
    print("discount factor (gamma) : ", gamma)
    print("--------------------------------------------------------------------------------------------")
    print("optimizer learning rate actor : ", lr_actor)
    print("optimizer learning rate critic : ", lr_critic)

    print("============================================================================================")

    agent = PPO(observation_shape,
                action_shape,
                lr_actor,
                lr_critic,
                gamma,
                K_epochs,
                eps_clip,
                action_std)


    print("============================================================================================")

    # To keep track of the progress
    print_running_reward = 0
    print_running_episodes = 0

    time_step = 0
    i_episode = 0
    counter = 0

    plot_episode = []
    plot_reward = []

    while time_step <= max_training_steps:
        obs, done = env.reset()
        current_ep_reward = 0
        for t in range(1, max_ep_len + 1):
            
            u = agent.select_action(obs)  # Get action under old_policy given state.
            if u > 1 or u < -1:
                print(u)
            action = unscaled_action(u)  # Unscale the action.
           
             
            obs, reward, done, _ = env.step(action)  # Apply the action to environment.
    

            # Append the reward and done flag to buffer for calculating Monte Carlo returns during updating phase.
            agent.buffer.rewards.append(reward)
            agent.buffer.dones.append(done)
            
            time_step += 1
            current_ep_reward += reward

            if time_step % update_timestep == 0:
                agent.update()

            if time_step % action_std_decay_freq == 0:
                # Decay standard deviation by 0.1.
                agent.decay_action_std(action_std_decay_rate, min_action_std)

            if time_step % print_freq == 0:
                # print average reward during 10_000 timesteps
                print_avg_reward = print_running_reward / print_running_episodes
                print_avg_reward = round(print_avg_reward, 2)

                print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, time_step,
                                                                                        print_avg_reward))

                print_running_reward = 0
                print_running_episodes = 0

            if time_step % save_model_freq == 0:

                # Save the model parameters for test phase and tracking performance.
                print("--------------------------------------------------------------------------------------------")
                print("Saving model at : " + checkpoint_path)
                agent.save(checkpoint_path)
                print("Model saved")
                counter += 1
                inter_checkpoint = directory + "PPO_{}_{}00K.pth".format(env_name, counter)
                print("--------------------------------------------------------------------------------------------")
                print("Model parameters to check for intermediate performance saving:.")
                print("saving model at : " + inter_checkpoint)
                agent.save(inter_checkpoint)
                if counter == 10:
                    print(f"Intermediate model saved for {counter}M")
                else:
                    print(f"Intermediate model saved for {counter}00K")
                print("--------------------------------------------------------------------------------------------")

            if done:
                break

        print_running_reward += current_ep_reward

        # plot reward per 10 episode.
        if i_episode % 10 == 0:
            plot_episode.append(i_episode)
            plot_reward.append(current_ep_reward)

        print_running_episodes += 1

        i_episode += 1

    env.close()

    print("============================================================================================")

    return plot_episode, plot_reward

In [12]:

n_episode, reward_episode = train()

Training Environment:DoubleInvPendulum
Environment initialized
--------------------------------------------------------------------------------------------
PPO update frequency : 2000 timesteps
PPO K epochs :  100
PPO epsilon clip :  0.2
discount factor (gamma) :  0.99
--------------------------------------------------------------------------------------------
optimizer learning rate actor :  0.0003
optimizer learning rate critic :  0.001
[1.3025029]
[-1.0793355]


C:\Users\Максимилиан\AppData\Roaming\Python\Python38\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


[1.0221399]
[1.7275804]
[-1.2659261]
[1.0589556]
[1.5166891]
[1.049843]
[1.3105762]
[-1.3961219]
[-1.4365257]
[-1.1729294]
[-1.0304153]
[1.5369287]
[-1.1822482]
[1.2807575]
[1.0735373]
[-1.195234]
[1.778472]
[1.2622848]
[-1.632945]
[1.0145168]
[1.4179134]
[-1.3264284]
[-1.3418809]
[-1.0625126]
[-1.2078282]
[1.3500963]
[-1.0036799]
[-1.1666349]
[1.0019734]
[-1.1961957]
[1.3622625]
[-1.512189]
[-1.2303703]
[-1.6332834]
[-1.7220224]
[-1.3772359]
[1.0429773]
[1.3201408]
[1.5751592]
[1.8113781]
[-1.5559007]
[1.2665517]
[-1.1550573]
[1.5191438]
[-2.134606]
[1.7971187]
[-1.0552176]
[-1.5633999]
[-1.7455708]
[1.0650923]
[-1.0013759]
[-1.5362703]
[1.1472696]
[1.0376921]
[-1.0131884]
[-1.149447]
[1.2576213]
[1.116726]
[1.9742234]
[1.2842555]
[1.1051769]
[-1.1618147]
[-1.4329447]
[1.4035187]
[-1.1254133]
[1.0038574]
[1.4956261]
[1.4167312]
[-1.1227685]
[1.6049358]
[-1.6219281]
[-1.04529]
[1.2014908]
[-1.196336]
[-1.027134]
[-1.198491]
[1.138653]
[1.2013947]
[-1.5009279]
[-1.4310951]
[1.1242049]
[

[1.095363]
[1.8948252]
[1.403856]
[-1.1551745]
[-1.213865]
[-1.0429785]
[1.1066171]
[1.1428282]
[1.6874892]
[-1.100004]


KeyboardInterrupt: 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir = 'PPO_plots'

In [ ]:
# Plot: Reward per every 10 episodes.
fig, ax = plt.subplots(figsize=(15, 10))
ax.plot(n_episode, reward_episode)
ax.set_title('Total number of episodes vs rewards per episode', fontsize=20)
ax.set_xlabel('Episode', fontsize=20)
ax.set_ylabel('Reward', fontsize=20)
plt.show()

In [ ]:
from matplotlib import animation
%matplotlib widget

state = state0

dt = 0.1
fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                     xlim=(-2, 2), ylim=(-2, 2))
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
energy_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)

def init():
    """initialize animation"""
    line.set_data([], [])
    #time_text.set_text('')
    energy_text.set_text('')
    return line


def animate(i):
    """perform animation step"""
    global state, dt
    state_t = torch.FloatTensor(state)
    u = agent.select_action(state_t)
    action = unscaled_action(u, action_low = -100, action_high = 100)
    state = get_next_state(state,u,dt)
    XY = state_to_coords(state)
    en = get_energy(state)
    
    line.set_data(XY[0],XY[1])
    energy_text.set_text(f'energy = {en}')
    return line,

ani = animation.FuncAnimation(fig, animate, frames=100,
                             interval=10, blit=True, init_func=init)

from IPython.display import HTML
HTML(ani.to_jshtml())